In [ ]:
df = pd.read_csv('movie_metadata.csv')

In [ ]:
df.head()

In [ ]:
import pandas as pd
df = pd.read_csv('movie_metadata.csv')

In [1]:
import pandas as pd
df = pd.read_csv('movie_metadata.csv')
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [2]:
import sqlite3
conn = sqlite3.connect('imdb_movies.db')
df.to_sql('movie_metadata', conn, if_exists='replace', index=False)

5043

In [3]:
query = '''
SELECT movie_title, imdb_score
FROM movie_metadata
WHERE imdb_score IS NOT NULL
ORDER BY imdb_score DESC
LIMIT 10
'''
pd.read_sql_query(query, conn)

,movie_title,imdb_score
0,Towering Inferno,9.5
1,The Shawshank Redemption,9.3
2,The Godfather,9.2
3,Dekalog,9.1
4,Dekalog,9.1
5,Kickboxer: Vengeance,9.1
6,The Dark Knight,9.0
7,The Godfather: Part II,9.0
8,Fargo,9.0
9,The Lord of the Rings: The Return of the King,8.9


In [5]:
# The prior query displays the top 10 movie titles with the highest score. 

# The following query will display categorical analysis and aggregation with 15 results. 

query = '''
SELECT genres, COUNT(*) AS movie_count
FROM movie_metadata
WHERE genres IS NOT NULL
GROUP BY genres
ORDER BY movie_count DESC
LIMIT 15
'''

pd.read_sql_query(query, conn)

,genres,movie_count
0,Drama,236
1,Comedy,209
2,Comedy|Drama,191
3,Comedy|Drama|Romance,187
4,Comedy|Romance,158
5,Drama|Romance,152
6,Crime|Drama|Thriller,101
7,Horror,71
8,Action|Crime|Drama|Thriller,68
9,Action|Crime|Thriller,65


In [7]:
# The following query displays average Score by Directory (min of 3 films). 

query = '''
SELECT director_name, COUNT(*) AS num_movies, AVG(imdb_score) AS avg_score
FROM movie_metadata
WHERE director_name IS NOT NULL AND imdb_score IS NOT NULL
GROUP BY director_name
HAVING COUNT(*) >= 3
ORDER BY avg_score DESC
LIMIT 10
'''

pd.read_sql_query(query, conn)

,director_name,num_movies,avg_score
0,Sergio Leone,4,8.475000
1,Christopher Nolan,8,8.425000
2,Pete Docter,3,8.233333
3,Hayao Miyazaki,4,8.225000
4,Quentin Tarantino,8,8.200000
5,Milos Forman,3,8.133333
6,Frank Capra,5,8.060000
7,Tony Kaye,3,8.033333
8,Stanley Kubrick,7,8.000000
9,David Lean,4,8.000000


In [8]:
# The following query lists average IMDb Score by Duration Category

query = '''
SELECT
 CASE
   WHEN duration < 90 THEN 'Short (<90 min)'
   WHEN duration BETWEEN 90 AND 120 THEN 'Medium (90-120 min)'
   WHEN duration > 120 THEN 'Long (>120 min)'
   ELSE 'Uknown'
 END AS duration_category,
 COUNT(*) AS movie_count,
 ROUND(AVG(imdb_score), 2) AS avg_score
FROM movie_metadata
WHERE duration IS NOT NULL AND imdb_score IS NOT NULL
GROUP BY duration_category
ORDER BY avg_score DESC
'''

pd.read_sql_query(query, conn)

,duration_category,movie_count,avg_score
0,Long (>120 min),1067,7.05
1,Medium (90-120 min),3169,6.31
2,Short (<90 min),792,6.15


In [9]:
# The following query lists Directors with the Most Films

query = '''
SELECT director_name, COUNT(*) AS film_count
FROM movie_metadata
WHERE director_name IS NOT NULL
GROUP BY director_name
ORDER BY film_count DESC
LIMIT 12
'''

pd.read_sql_query(query, conn)

,director_name,film_count
0,Steven Spielberg,26
1,Woody Allen,22
2,Martin Scorsese,20
3,Clint Eastwood,20
4,Ridley Scott,17
5,Tim Burton,16
6,Steven Soderbergh,16
7,Spike Lee,16
8,Renny Harlin,15
9,Oliver Stone,14
